# Deploying Machine Learning Models on GCP Kubernetes

<img src="images/deploy-graph.png" alt="predictor with canary" title="ml graph"/>

## Prerequisites
* You need a running GCP cluster with kubernetes>1.8 with kubectl configured to use.
* If you wish to test the JSON schema checks you will need presently to enbale "alpha features" for your cluster (Jan 2018).
* A clone of the latest seldon core
- [python grpc tools](https://grpc.io/docs/quickstart/python.html)

## Install helm

In [1]:
!kubectl -n kube-system create sa tiller
!kubectl create clusterrolebinding tiller --clusterrole cluster-admin --serviceaccount=kube-system:tiller
!helm init --service-account tiller

serviceaccount "tiller" created
clusterrolebinding "tiller" created
$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.
Happy Helming!


## Start Seldon-Core

In [3]:
!helm install ../helm-charts/seldon-core-crd --name seldon-core-crd

NAME:   seldon-core-crd
LAST DEPLOYED: Wed Feb 28 13:02:18 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  1s


NOTES:
NOTES: TODO




In [1]:
!helm install ../helm-charts/seldon-core --name seldon-core \
        --set cluster_manager.rbac=true \
        --set apife.enabled=true \
        --set engine.image.tag=0.1.6_SNAPSHOT_loadtest
        

NAME:   seldon-core
LAST DEPLOYED: Thu Mar  1 11:24:13 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/Service
NAME              TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)                        AGE
seldon-apiserver  NodePort   10.3.251.111  <none>       8080:31511/TCP,5000:30139/TCP  1s
redis             ClusterIP  10.3.242.20   <none>       6379/TCP                       1s

==> v1/ServiceAccount
NAME    SECRETS  AGE
seldon  1        1s

==> v1beta1/ClusterRoleBinding
NAME    AGE
seldon  1s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
redis-df886d999-k99bh  0/1    ContainerCreating  0         1s

==> v1beta1/Deployment
NAME                    DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-apiserver        1        0        0           0          1s
seldon-cluster-manager  1        0        0           0          1s
redis                   1        1        1           0          1s


NOTES:
NOTES: TODO




## Set up REST and GRPC methods

Wait for API server to be given an external IP

In [6]:
!kubectl get svc seldon-apiserver

NAME               TYPE       CLUSTER-IP    EXTERNAL-IP   PORT(S)                         AGE
seldon-apiserver   NodePort   10.3.246.87   <none>        8080:32665/TCP,5000:32324/TCP   20m


In [3]:
!cp ../proto/prediction.proto ./proto
!python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto

Setup python code to do REST and gRPC requests. **Only run this when the LoadBalancer created by GCP for the seldon-apife is running**

In [4]:
import requests
from requests.auth import HTTPBasicAuth
from proto import prediction_pb2
from proto import prediction_pb2_grpc
import grpc
try:
    from commands import getoutput # python 2
except ImportError:
    from subprocess import getoutput # python 3

def get_token():
    payload = {'grant_type': 'client_credentials'}
    response = requests.post(
                "http://0.0.0.0:8080/oauth/token",
                auth=HTTPBasicAuth('oauth-key', 'oauth-secret'),
                data=payload)
    token =  response.json()["access_token"]
    return token

def rest_request():
    token = get_token()
    headers = {'Authorization': 'Bearer '+token}
    payload = {"data":{"names":["a","b"],"tensor":{"shape":[2,2],"values":[0,0,1,1]}}}
    response = requests.post(
                "http://0.0.0.0:8080/api/v0.1/predictions",
                headers=headers,
                json=payload)
    print(response.text)
    
def grpc_request():
    token = get_token()
    datadef = prediction_pb2.DefaultData(
            names = ["a","b"],
            tensor = prediction_pb2.Tensor(
                shape = [3,2],
                values = [1.0,1.0,2.0,3.0,4.0,5.0]
                )
            )
    request = prediction_pb2.SeldonMessage(data = datadef)
    channel = grpc.insecure_channel("0.0.0.0:5000")
    stub = prediction_pb2_grpc.SeldonStub(channel)
    metadata = [('oauth_token', token)]
    response = stub.Predict(request=request,metadata=metadata)
    print(response)


## Normal Operation

### Create Seldon Deployment

In [10]:
!kubectl create -f resources/model.json

seldondeployment "seldon-deployment-example" created


In [11]:
!kubectl get seldondeployments

NAME                        AGE
seldon-deployment-example   7s


Get the status of the SeldonDeployment. **When ready the replicasAvailable should be 1**.

In [15]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}'

map[predictorStatus:[map[replicas:1 replicasAvailable:1 name:test-deployment-fx-market-predictor]]]

### Get Predictions

In [6]:
# REST Request
rest_request()

{
  "status": {
    "code": 0,
    "info": "",
    "reason": "",
    "status": "SUCCESS"
  },
  "meta": {
    "puid": "gjk245gd3iu8kk451q6n8qa7qt",
    "tags": {
    },
    "routing": {
    }
  },
  "data": {
    "names": ["class0", "class1", "class2"],
    "tensor": {
      "shape": [1, 3],
      "values": [0.1, 0.9, 0.5]
    }
  }
}


In [ ]:
# GRPC Request
grpc_request()

## Load test

In [ ]:
!kubectl create clusterrolebinding default-admin --clusterrole=cluster-admin --serviceaccount=default:default

In [2]:
!helm install ../helm-charts/seldon-core-analytics --name seldon-core-analytics \
    --set grafana_prom_admin_password=password \
    --set persistence.enabled=false

NAME:   seldon-core-analytics
LAST DEPLOYED: Thu Mar  1 11:24:29 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ConfigMap
NAME                       DATA  AGE
alertmanager-server-conf   1     8s
grafana-import-dashboards  5     8s
prometheus-rules           4     8s
prometheus-server-conf     1     8s

==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           8s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           1          8s
grafana-prom-deployment  1        1        1           1          8s
prometheus-deployment    1        1        1           0          7s

==> v1/Service
NAME                      TYPE       CLUSTER-IP    EXTERNAL-IP  PORT(S)       AGE
alertmanager              ClusterIP  10.3.241.37   <none>       80/TCP        8s
grafana-prom              NodePort   10.3.251.167  <none>       80:32474/TCP  8

In [9]:
!helm install ../helm-charts/seldon-core-loadtesting --name loadtest  \
    --set locust.host=http://loadtest:8000 \
    --set oauth.enabled=false \
    --set oauth.key=oauth-key \
    --set oauth.secret=oauth-secret \
    --set locust.hatchRate=1 \
    --set locust.clients=5 \
    --set loadtest.sendFeedback=0 \
    --set locust.minWait=0 \
    --set locust.maxWait=0 \
    --set replicaCount=2

NAME:   loadtest
LAST DEPLOYED: Thu Mar  1 11:43:08 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ReplicationController
NAME             DESIRED  CURRENT  READY  AGE
locust-slave-1   2        2        0      0s
locust-master-1  1        1        0      0s

==> v1/Service
NAME             TYPE      CLUSTER-IP   EXTERNAL-IP  PORT(S)                                       AGE
locust-master-1  NodePort  10.3.246.36  <none>       5557:30614/TCP,5558:30396/TCP,8089:31666/TCP  0s

==> v1/Pod(related)
NAME                   READY  STATUS             RESTARTS  AGE
locust-slave-1-84s8x   0/1    ContainerCreating  0         0s
locust-slave-1-dldlv   0/1    ContainerCreating  0         0s
locust-master-1-8b9wp  0/1    ContainerCreating  0         0s




In [10]:
!helm delete loadtest --purge

release "loadtest" deleted


## Update Deployment with Canary

In [ ]:
!kubectl apply -f resources/model_with_canary.json -n seldon

Check the status of the deployments. Note: **Might need to run several times until replicasAvailable is 1 for both predictors**.

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' -n seldon

In [ ]:
rest_request()

In [ ]:
grpc_request()

## Tear Down

In [ ]:
!kubectl delete -f resources/model.json

In [ ]:
!helm delete seldon-core --purge

In [ ]:
!helm delete seldon-core-crd --purge